In [1]:
import candas as can
import numpy as np
import pandas as pd
from matplotlib.colors import Normalize
import matplotlib as mpl
mpl.use('Cairo')  # for saving SVGs that Affinity Designer can parse
import matplotlib.pyplot as plt
import seaborn as sns

from candas.test import QuantStudio
from candas.learn import ParameterSet
from candas.lims import Librarian, library

base_pth, code_pth, data_pth, _, fig_pth = can.utils.setup_paths(make_missing=False)
plt.style.use('style.mplstyle')

%config InlineBackend.figure_format = 'retina'

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
from tinydb import TinyDB, Query
db_file = base_pth / 'Data' / 'JG071B Reaction Specifications.json'
rxn_db = TinyDB(db_file)
JG071B_rxns = pd.DataFrame([rxn['oligos'] | {'Sample': rxn['name']} for rxn in rxn_db.all()])

lib = Librarian().load(library).open()

In [3]:
cmax=50

file = data_pth / 'JG071B Tripartite sweep GBP6.xlsx'
JG071B = (
    QuantStudio(file, 'JG071B')
    .import_data()
    .format_reactions()
    .index_reactions()
    .subtract_background()
    .drop(lambda df: df['Well'].isin([54, 281]))
    .normalize_reactions(cmax=cmax, method='max')
    .invert_fluorophore('HEX')
)

JG071B.reactions.data = JG071B.reactions.data.merge(JG071B_rxns)
for oligo in ['S036.5', 'S057.3.2', 'S057.4.2']:
    JG071B.reactions.data['lg10 ' + oligo] = np.log10(JG071B.reactions.data[oligo])
    
JG071B.extract_endpoints(cmax=cmax, name='FAM-HEX', method='diff')


In [4]:
width = 3.65
height = 2.187
figsize = (width, height)
spotsize = 4**2
ticklabelsize = 6
labelsize = 6
linewidth = 1

mar_l = 0.325
mar_r = 0.22
mar_t = 0.22
mar_b = 0.275
ax_space = 0.075

def format_sns_axes(g, xticks, xlim, ylim=None):
    fig = g.figure
    fig.set_size_inches(figsize)
    
    # g.set_titles('')

    ylim = ylim or [-1.1, 1.1]

    plt.setp(
        g.axes,
        ylim=ylim,
        xticks=xticks,
        xlim=xlim,
        # xlabel="",
        # xticklabels="",
        title="",
    )
    for i, ax in enumerate(g.axes.flat):
        ax.tick_params(axis="both", labelsize=ticklabelsize, length=1, width=0.6)
        ax.set_ylabel(ax.get_ylabel(), fontsize=labelsize, labelpad=0)
        ax.set_xlabel(ax.get_xlabel(), fontsize=labelsize, labelpad=2)
        if i > 0:
            ax.yaxis.set_tick_params(length=0)
        ax.axhline(0, color="k", linestyle="-", linewidth=0.5, zorder=-1)


    for (row, col), ax in g.axes_dict.items():
        ax.set_title('')
            
        if col==8.0:
            ax2 = ax.twinx()
            ax2.set_yticks([])
            ax2.set_ylabel(f'{row:.1f}', va='bottom', fontsize=labelsize, rotation=270, labelpad=1)
            if row==5.0:
                ax2.set_ylabel(f'lg10 Copies Competitor 2\n{row:.1f}', va='bottom', fontsize=labelsize, rotation=270, labelpad=1)
        
        if row==8.0:
            ax.set_title(f'{col:.1f}', fontsize=labelsize, pad=2)
            if col==5.0:
                ax.set_title(f'lg10 Copies Competitor 1\n{col:.1f}', fontsize=labelsize, pad=2)


    bbox = ax.get_window_extent().transformed(ax.figure.dpi_scale_trans.inverted())
    ax_width, ax_height = bbox.width, bbox.height

    plt.subplots_adjust(
        left=mar_l / width,
        right=1 - mar_r / width,
        top=1 - mar_t / height,
        bottom=mar_b / height,
        wspace=ax_space/ax_width,
        hspace=ax_space/ax_height,
    )

In [5]:
hue = 'lg10 S036.5'
extent = np.max(np.abs(JG071B.reactions.wide[hue]))
norm = mpl.colors.Normalize(vmin=JG071B.reactions.wide[hue].min(), vmax=JG071B.reactions.wide[hue].max())

g = sns.relplot(data = JG071B.reactions.data.query('Cycle<=@cmax'),
                x='Cycle', y='Fluorescence', col='lg10 S057.3.2', row='lg10 S057.4.2', units='Reaction', hue = hue,
                legend = False, palette='ch:0_r', hue_norm=norm, kind='line', estimator=None, linewidth=linewidth,
                row_order = JG071B.reactions.data['lg10 S057.4.2'].unique()[::-1],
                facet_kws={
                    'margin_titles': False,
                    'despine': False})

g.refline(y=0, color='k', linestyle='-', linewidth=0.5)
format_sns_axes(g, xticks=[0, 10, 20, 30, 40, 50], xlim=[-2.5, 52.5])

plt.savefig(fig_pth / 'JG071B GBP6 Tripartite Curves.png', dpi=300, transparent=True);
plt.savefig(fig_pth / 'JG071B GBP6 Tripartite Curves.svg', dpi=300, transparent=True);

In [6]:
endpoints = JG071B.endpoints

hue = 'FAM-HEX'
extent = np.max(np.abs(endpoints[hue]))
norm = mpl.colors.Normalize(vmin=-extent, vmax=+extent)

g = sns.relplot(data = endpoints[endpoints.Reporter=='FAM'],
                x='lg10 S036.5', y=hue, col='lg10 S057.3.2', row='lg10 S057.4.2', units='Reaction', hue=hue, hue_norm=norm,
                legend = False, palette='coolwarm', kind='scatter', estimator=None, s=spotsize,
                row_order = endpoints['lg10 S057.4.2'].unique()[::-1],
                facet_kws={
                    'margin_titles': False,
                    'despine': False})

g.refline(y=0, color='k', linestyle='-', linewidth=0.5)
format_sns_axes(g, xticks=[2, 4, 6, 8], xlim=[0.5, 8.5])

plt.savefig(fig_pth / 'JG071B GBP6 Tripartite Endpoints.png', dpi=300, transparent=True);
plt.savefig(fig_pth / 'JG071B GBP6 Tripartite Endpoints.svg', dpi=300, transparent=True);

In [7]:
db_file = data_pth / 'JG071A Reaction Specifications.json'
rxn_db = TinyDB(db_file)
JG071A_rxns = pd.DataFrame([rxn['oligos'] | {'Sample': rxn['name']} for rxn in rxn_db.all()])

file = data_pth / 'JG071A ARG1 Tripartite.xlsx'
JG071A = (
    QuantStudio(file, 'JG071A')
    .import_data()
    .format_reactions()
    .index_reactions()
    .subtract_background()
    .drop(lambda df: df['Well'].isin([281]))
    .normalize_reactions(cmax=cmax, method='max')
    .invert_fluorophore('HEX')
)

JG071A.reactions.data = JG071A.reactions.data.merge(JG071A_rxns)
for oligo in ['S036.0', 'S057.0.1', 'S057.1.2']:
    JG071A.reactions.data['lg10 ' + oligo] = np.log10(JG071A.reactions.data[oligo])
    
JG071A.extract_endpoints(cmax=cmax, name='FAM-HEX', method='diff')


In [8]:
hue = 'lg10 S036.0'
extent = np.max(np.abs(JG071A.reactions.wide[hue]))
norm = mpl.colors.Normalize(vmin=JG071A.reactions.wide[hue].min(), vmax=JG071A.reactions.wide[hue].max())

g = sns.relplot(data = JG071A.reactions.data.query('Cycle<=@cmax'),
                x='Cycle', y='Fluorescence', col='lg10 S057.0.1', row='lg10 S057.1.2', units='Reaction', hue = hue,
                legend = False, palette='ch:0_r', hue_norm=norm, kind='line', estimator=None, linewidth=linewidth,
                row_order = JG071A.reactions.data['lg10 S057.1.2'].unique()[::-1],
                facet_kws={
                    'margin_titles': False,
                    'despine': False})

g.refline(y=0, color='k', linestyle='-', linewidth=0.5)
format_sns_axes(g, xticks=[0, 10, 20, 30, 40, 50], xlim=[-2.5, 52.5])

plt.savefig(fig_pth / 'JG071A ARG1 Tripartite Curves.png', dpi=300, transparent=True);
plt.savefig(fig_pth / 'JG071A ARG1 Tripartite Curves.svg', dpi=300, transparent=True);

In [9]:
endpoints = JG071A.endpoints

hue = 'FAM-HEX'
extent = np.max(np.abs(endpoints[hue]))
norm = mpl.colors.Normalize(vmin=-extent, vmax=+extent)

g = sns.relplot(data = endpoints[endpoints.Reporter=='FAM'],
                x='lg10 S036.0', y=hue, col='lg10 S057.0.1', row='lg10 S057.1.2', units='Reaction', hue=hue, hue_norm=norm,
                legend = False, palette='coolwarm', kind='scatter', estimator=None, s=spotsize,
                row_order = endpoints['lg10 S057.1.2'].unique()[::-1],
                facet_kws={
                    'margin_titles': False,
                    'despine': False})

g.refline(y=0, color='k', linestyle='-', linewidth=0.5)
format_sns_axes(g, xticks=[2, 4, 6, 8], xlim=[0.5, 8.5])

plt.savefig(fig_pth / 'JG071A ARG1 Tripartite Endpoints.png', dpi=300, transparent=True);
plt.savefig(fig_pth / 'JG071A ARG1 Tripartite Endpoints.svg', dpi=300, transparent=True);